In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

### Sequential chain -- LLMChain

In [6]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [7]:
dictionary_template = PromptTemplate(
    input_variables = ["word"],
    template="Give me a one line definition of {word}. Response should not contain the word - {word} itself, only the meaning."
)

In [8]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')

In [10]:
chain = dictionary_template | llm | StrOutputParser()

In [11]:
chain.invoke({"word":"Onomatopoeia"})

"It's a word that imitates the natural sound of something. "

In [13]:
guesser_template = PromptTemplate(
    input_variables=["definition"],
    template="Give me an English word which has the following meaning- {definition}."
)

In [12]:
guesser_llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')

In [14]:
chain_2 = guesser_template.pipe(guesser_llm).pipe(StrOutputParser())

In [15]:
chain_2.invoke("A word  that imitates the sound it represents.")

"The word you're looking for is **onomatopoeia**. \n\nOnomatopoeia refers to words that phonetically resemble the sounds they describe. Examples include: \n\n* **Bang**\n* **Crash**\n* **Meow**\n* **Sizzle** "

In [16]:
seq_chain = {"definition": chain} | chain_2

In [17]:
seq_chain.invoke({"word":"Onomatopoeia"})

"The word you're looking for is **onomatopoeia**. \n\nOnomatopoeia refers to words that phonetically imitate or suggest the sound that they are describing.  \n\nHere are some examples:\n\n* **Bang**\n* **Crash**\n* **Sizzle**\n* **Meow**\n* **Woof** "

In [18]:
seq_chain.invoke({"word":"Sesquipedalian"})

'**Sesquipedalian** \n\nIronically, "sesquipedalian" itself is a long word! It\'s used to describe language that is characterized by the use of long words, often in a showy or pretentious way. '

In [19]:
seq_chain.invoke({"word":"Innate"})

'**Innate** '

In [21]:
seq_chain.invoke({"word":"Philanthrophy"})

'**Philanthropy** \n\nWhile other words like "charity," "altruism," or "generosity" touch upon aspects of the definition, "philanthropy" encompasses the comprehensive act of giving with the specific aim of improving the well-being of others. '